In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0
sys.version_info(major=3, minor=7, micro=4, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.2
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0
tensorflow_core.keras 2.2.4-tf


In [2]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
print(housing.DESCR)
print(housing.data.shape)
print(housing.target.shape)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    housing.data, housing.target, random_state = 7)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, random_state = 11)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [5]:
X_train.shape

(11610, 8)

In [6]:
X_train.shape[1:]

(8,)

In [7]:
# dense layer
layer = tf.keras.layers.Dense(100)
layer = tf.keras.layers.Dense(100, input_shape=(None, 5))
layer(tf.zeros([10, 5]))

# layer.variables
# x * W + b
layer.trainable_variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 100) dtype=float32, numpy=
 array([[-0.20634973, -0.15688755, -0.2022606 ,  0.15898477,  0.09511502,
         -0.0490035 ,  0.10637717, -0.23766632, -0.11466946,  0.0819131 ,
         -0.00297675, -0.01546703, -0.01701587,  0.20919316,  0.20300321,
         -0.2093059 , -0.20762968, -0.20207594,  0.0462084 , -0.11583884,
          0.06614111,  0.09055419,  0.09322514, -0.22951041,  0.02342956,
          0.05513977,  0.07193007,  0.13190778,  0.2215466 ,  0.01785178,
         -0.03869256, -0.23828925,  0.05751626, -0.06896442, -0.21920523,
         -0.23123102,  0.09649007, -0.19421126,  0.01169278, -0.08160996,
         -0.12790316, -0.00991842,  0.01243638,  0.12903617, -0.11284922,
         -0.0678575 , -0.08284083,  0.21191747,  0.14014627, -0.2284243 ,
          0.01757313,  0.0759659 , -0.15247327, -0.00298136, -0.22997336,
          0.06487341,  0.00569507,  0.13789771,  0.18328975,  0.23238446,
          0.17198698,  0.20278601,  0.2165

In [11]:
class CustomizedDenseLayer(keras.layers.Layer):
    def __init__(self,units,activation,**kwargs):
        self.units = units
        self.activation = keras.layers.Activation(activation)
        super(CustomizedDenseLayer,self).__init__(**kwargs)
    
    def build(self , input_shape):
        """构建所需要的参数"""
        # x * w + b 
        # input_shape:[None, a] 
        # w:[a,b] 
        # output_shape: [None, b]
        self.kernel = self.add_weight(
            name = 'kernel'
            , shape = (input_shape[1],self.units) # 相当于矩阵乘法 a,b * b,c = a,c
            , initializer = 'uniform' # 如何初始化参数矩阵 ， 随机化分布，uniform
            , trainable=True # 可被训练， 如果不变设置为 False
        )
        self.bias = self.add_weight(
            name = 'bias'
            , shape = (self.units)
            , initializer = 'zeros'
            , trainable=True
        )
        super(CustomizedDenseLayer,self).build(input_shape)
        
    def call(self):
        '''完成正向计算'''
        return self.activation(x @ self.kernel + bias)
        

In [ ]:
# tf.nn.softplus : log(1+e^x)
customized_softplus = keras.layers.Lambda(lambda x : tf.nn.softplus(x))
print(customized_softplus([-10., -5., 0., 5., 10.]))

In [ ]:
def mse(y_true,y_pred):
    return tf.reduce_mean(tf.square(y_pred-y_true))
model = keras.models.Sequential([
    keras.layers.Dense(30,activation='relu',input_shape=X_train.shape[1:])
    ,keras.layers.Dense(1)
])
model.summary()
call_backs=[
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3)
]
model.compile(loss=mse,optimizer=keras.optimizers.SGD(),metric=keras.metrics.mean_squared_error)

In [ ]:
history_model=model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=50,callbacks=call_backs)

In [ ]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8, 5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()
plot_learning_curves(history)